## GPU-accelerated approximate k-mer counting using a bloom filter
#### Integrating Project | G01 
##### Members: Bawa, Lejano, Roxas

#### CUDA Setup

In [1]:
import os
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

#### Bloom filter setup

In [2]:
!git clone https://github.com/jvirkki/libbloom.git

fatal: destination path 'libbloom' already exists and is not an empty directory.


In [3]:
# build libbloom
!cd libbloom && make

make: Nothing to be done for 'all'.


In [4]:
# verify build
!ls libbloom


bloom.c  build	    docs     Makefile  murmur2
bloom.h  ChangeLog  LICENSE  misc      README


#### Hash table setup

In [5]:
!wget -q https://raw.githubusercontent.com/troydhanson/uthash/master/src/uthash.h -O uthash.h

In [6]:
!gcc test_bloom.c -Ilibbloom libbloom/build/libbloom.a -lm -o test_bloom.out


#### Build helper files

In [7]:
%%writefile fasta_reader.h
#ifndef FASTA_READER_H
#define FASTA_READER_H

char* load_sequence(const char* filename);

#endif

Overwriting fasta_reader.h


In [8]:
%%writefile fasta_reader.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include "fasta_reader.h"

char* load_sequence(const char* filename) {
    FILE* fp = fopen(filename, "r");
    if (!fp) {
        perror("Failed to open file");
        return NULL;
    }

    size_t capacity = 1024;
    size_t length = 0;
    char* sequence = malloc(capacity);

    char line[1024];
    while (fgets(line, sizeof(line), fp)) {
        if (line[0] == '>') continue;
        for (int i = 0; line[i]; i++) {
            char c = toupper(line[i]);
            if (c == 'A' || c == 'T' || c == 'G' || c == 'C' || c == 'U') {
                if (length + 1 >= capacity) {
                    capacity *= 2;
                    sequence = realloc(sequence, capacity);
                }
                sequence[length++] = c;
            }
        }
    }
    fclose(fp);
    sequence[length] = '\0'; // Null-terminate string
    return sequence;
}


Overwriting fasta_reader.c


In [9]:
%%writefile ground_truth.h
#ifndef GROUND_TRUTH_H
#define GROUND_TRUTH_H

#include <stddef.h>
#include "uthash.h"

typedef struct KmerEntry {
    char kmer[64];
    int count;
    UT_hash_handle hh;
} KmerEntry;


void get_ground_truth(const char *seq, int k, size_t length, size_t *total_unique, size_t *true_non_singletons, KmerEntry **truth_table);
void free_truth_table(KmerEntry *truth_table);

#endif

Overwriting ground_truth.h


In [10]:
%%writefile ground_truth.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "fasta_reader.h"
#include "ground_truth.h"
#include "uthash.h"

void get_ground_truth(const char *seq, int k, size_t length, size_t *total_unique, size_t *true_non_singletons, KmerEntry **truth_table) {
    *truth_table = NULL;
    
    if (length < (size_t)k) {
        printf("Sequence shorter than k-mer length.\n");
        return;
    }

    for (size_t i = 0; i <= length - k; i++) {
        char kmer[k + 1];
        strncpy(kmer, seq + i, k);
        kmer[k] = '\0';

        KmerEntry *entry;
        HASH_FIND_STR(*truth_table, kmer, entry);
        if (entry) {
            entry->count++;
        } else {
            entry = malloc(sizeof(KmerEntry));
            strcpy(entry->kmer, kmer);
            entry->count = 1;
            HASH_ADD_STR(*truth_table, kmer, entry);
        }
    }

    KmerEntry *s, *tmp;
    HASH_ITER(hh, *truth_table, s, tmp) {
        (*total_unique)++;
        if (s->count < 2) {
            HASH_DEL(*truth_table, s);
            free(s);
        } else {
            (*true_non_singletons)++;
        }
    }

    printf("Sequence length: %zu\n", length);
    printf("Total unique k-mers: %zu\n", *total_unique);
    printf("True non-singletons: %zu\n", *true_non_singletons);
    printf("\n");
}

void free_truth_table(KmerEntry *truth_table) {
    KmerEntry *s, *tmp;
    HASH_ITER(hh, truth_table, s, tmp) {
        HASH_DEL(truth_table, s);
        free(s);
    }
}


Overwriting ground_truth.c


In [11]:
%%writefile serialized.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "libbloom/bloom.h"
#include "uthash.h"
# include "fasta_reader.h"
#include "ground_truth.h"
#include <string.h>

// Candidate List

typedef struct {
    char **items;
    size_t size;
    size_t capacity;
} CandidateList;

void init_candidate_list(CandidateList *list) {
    list->size = 0;
    list->capacity = 1024;
    list->items = malloc(list->capacity * sizeof(char*));
}

void add_candidate(CandidateList *list, const char *kmer) {
    if (list->size >= list->capacity) {
        list->capacity *= 2;
        list->items = realloc(list->items, list->capacity * sizeof(char*));
    }
    list->items[list->size++] = strdup(kmer);
}

void free_candidate_list(CandidateList *list) {
    for (size_t i = 0; i < list->size; i++) free(list->items[i]);
    free(list->items);
}

// Phase 1

void phase1_filter(const char *sequence, int k, size_t entries, double error_rate, CandidateList *out_candidates) {
    struct bloom filter;
    bloom_init(&filter, entries, error_rate);

    init_candidate_list(out_candidates);

    size_t length = strlen(sequence);
    for (size_t i = 0; i <= length - k; i++) {
        char kmer[k + 1];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        if (bloom_check(&filter, kmer, k)) {
            add_candidate(out_candidates, kmer);
        }
        bloom_add(&filter, kmer, k);
    }

    bloom_free(&filter);
}

// Phase 2

typedef struct {
    char kmer[64];
    int count;
    UT_hash_handle hh;
} KmerCount;

void phase2_count(const char *sequence, CandidateList *candidates, int k) {
    KmerCount *counts = NULL;
    size_t length = strlen(sequence);

    for (size_t i = 0; i <= length - k; i++) {
        char kmer[k + 1];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        int is_candidate = 0;
        for (size_t j = 0; j < candidates->size; j++) {
            if (strcmp(kmer, candidates->items[j]) == 0) {
                is_candidate = 1;
                break;
            }
        }
        if (!is_candidate) continue;

        KmerCount *entry;
        HASH_FIND_STR(counts, kmer, entry);
        if (entry) entry->count++;
        else {
            entry = malloc(sizeof(KmerCount));
            strcpy(entry->kmer, kmer);
            entry->count = 1;
            HASH_ADD_STR(counts, kmer, entry);
        }
    }

    KmerCount *s, *tmp;
    HASH_ITER(hh, counts, s, tmp) {
        HASH_DEL(counts, s);
        free(s);
    }
}

int main() {
    // initialize variables
    char* seq = load_sequence("Homo_Sapiens_Ch17.fasta");
    if (!seq) return 1;

    int k = 21;
    double error_rate = 0.01;

    // varying bloom filter sizes
    // Larger entries → more memory → fewer false positives.
    // Smaller entries → smaller memory → more false positives.
    size_t length = strlen(seq);
    size_t estimated_unique_kmers = length - k + 1;
    size_t filter_sizes[7];
    filter_sizes[0] = 10000;
    filter_sizes[1] = 50000;
    filter_sizes[2] = 100000;
    filter_sizes[3] = 2500000;
    filter_sizes[4] = 5000000;
    filter_sizes[5] = 10000000;
    filter_sizes[6] = estimated_unique_kmers;
    int num_sizes = sizeof(filter_sizes) / sizeof(filter_sizes[0]);

    // timer variables
    clock_t p1_start, p2_start, p1_end, p2_end;
    double p1_time_taken, p2_time_taken, total_time_taken;

    // get ground-truth false positives
    KmerEntry *truth_table = NULL;
    size_t total_unique = 0, true_non_singletons = 0;
    get_ground_truth(seq, k, length, &total_unique, &true_non_singletons, &truth_table);

    printf("Note: false positive rate = (candidates - true non-singletons) / true non-singletons\n");
    printf("%-12s %-12s %-12s %-14s %-12s %-15s %-18s %-16s\n",
        "BloomSize", "Phase1_ms", "Phase2_ms", "TotalTime_ms",
        "Candidates", "FalsePositives", "FalsePositiveRate", "FalseNegatives");

    for (int i = 0; i < num_sizes; i++) {
        size_t entries = filter_sizes[i];

        CandidateList candidates;

        p1_start = clock();
        phase1_filter(seq, k, entries, error_rate, &candidates);
        p1_end = clock();

        p2_start = clock();
        phase2_count(seq, &candidates, k);
        p2_end = clock();

        p1_time_taken = ((double)(p1_end-p1_start))*1E3/CLOCKS_PER_SEC;
        p2_time_taken = ((double)(p2_end-p2_start))*1E3/CLOCKS_PER_SEC;
        total_time_taken = p1_time_taken + p2_time_taken;

        // false positives
        size_t false_positives = 0;
        if (candidates.size > true_non_singletons) {
            false_positives = candidates.size - true_non_singletons;
        }

        double fp_rate = true_non_singletons > 0 ? (double)false_positives / (double)true_non_singletons : 0.0;

        // false negatives
        size_t false_negatives = 0;
        KmerEntry *t, *tmp;
        HASH_ITER(hh, truth_table, t, tmp) {
            int found = 0;
            for (size_t j = 0; j < candidates.size; j++) {
                if (strcmp(t->kmer, candidates.items[j]) == 0) {
                    found = 1;
                    break;
                }
            }
            if (!found)
                false_negatives++;
        }
        
    
        printf("%-12zu %-12.2f %-12.2f %-14.2f %-12zu %-15zu %-18.4f %-16zu\n",
           entries, p1_time_taken, p2_time_taken, total_time_taken,
           candidates.size, false_positives, fp_rate, false_negatives);

        free_candidate_list(&candidates);        
    }
    
    free_truth_table(truth_table);
    free(seq);
    return 0;
}

Overwriting serialized.c


DELETE LATER, but notes:

Small Bloom size → extremely high false positive rate → tons of Phase 2 work → very slow.

Bloom size close to real number of unique k-mers → optimal.

In [12]:
!gcc serialized.c fasta_reader.c ground_truth.c -Ilibbloom libbloom/build/libbloom.a -lm -o serialized.out
!./serialized.out

Sequence length: 81188
Total unique k-mers: 76575
True non-singletons: 2199

Note: false positive rate = (candidates - true non-singletons) / true non-singletons
BloomSize    Phase1_ms    Phase2_ms    TotalTime_ms   Candidates   FalsePositives  FalsePositiveRate  FalseNegatives  
10000        22.82        22941.92     22964.74       46092        43893           19.9604            0               
50000        19.59        2467.80      2487.39        5526         3327            1.5130             0               
100000       25.37        2370.20      2395.57        4639         2440            1.1096             0               
2500000      111.50       2149.14      2260.65        4593         2394            1.0887             0               
5000000      194.20       1783.77      1977.96        4593         2394            1.0887             0               
10000000     202.50       1688.23      1890.73        4593         2394            1.0887             0               
81168